# Boston House Predicate

In [1]:
#TODO：
'''
The dataframe has 506 rows and 14 columns.And data refers to crime,rooms,nox(Nitrogen index),etc
And our goal is to predict Boston house price MEDV（中位数）

And our scoring is based on RMSE.
'''

'\nThe dataframe has 506 rows and 14 columns.And data refers to crime,rooms,nox(Nitrogen index),etc\nAnd our goal is to predict Boston house price MEDV（中位数）\n'

In [3]:
import keras
from keras.datasets import boston_housing
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import RMSprop
import numpy as np

In [4]:
(train_data,train_target),(test_data,test_target)=boston_housing.load_data()

57344/57026 [==============================] - 0s 5us/step


In [5]:
train_data.shape

(404, 13)

In [6]:
test_data.shape

(102, 13)

In [9]:
train_target #先看一眼，大概结果是如下结果，单位是 千美元

array([15.2, 42.3, 50. , 21.1, 17.7, 18.5, 11.3, 15.6, 15.6, 14.4, 12.1,
       17.9, 23.1, 19.9, 15.7,  8.8, 50. , 22.5, 24.1, 27.5, 10.9, 30.8,
       32.9, 24. , 18.5, 13.3, 22.9, 34.7, 16.6, 17.5, 22.3, 16.1, 14.9,
       23.1, 34.9, 25. , 13.9, 13.1, 20.4, 20. , 15.2, 24.7, 22.2, 16.7,
       12.7, 15.6, 18.4, 21. , 30.1, 15.1, 18.7,  9.6, 31.5, 24.8, 19.1,
       22. , 14.5, 11. , 32. , 29.4, 20.3, 24.4, 14.6, 19.5, 14.1, 14.3,
       15.6, 10.5,  6.3, 19.3, 19.3, 13.4, 36.4, 17.8, 13.5, 16.5,  8.3,
       14.3, 16. , 13.4, 28.6, 43.5, 20.2, 22. , 23. , 20.7, 12.5, 48.5,
       14.6, 13.4, 23.7, 50. , 21.7, 39.8, 38.7, 22.2, 34.9, 22.5, 31.1,
       28.7, 46. , 41.7, 21. , 26.6, 15. , 24.4, 13.3, 21.2, 11.7, 21.7,
       19.4, 50. , 22.8, 19.7, 24.7, 36.2, 14.2, 18.9, 18.3, 20.6, 24.6,
       18.2,  8.7, 44. , 10.4, 13.2, 21.2, 37. , 30.7, 22.9, 20. , 19.3,
       31.7, 32. , 23.1, 18.8, 10.9, 50. , 19.6,  5. , 14.4, 19.8, 13.8,
       19.6, 23.9, 24.5, 25. , 19.9, 17.2, 24.6, 13

由概率论知识，我们进行标准化处理：$X_t =\frac{X-mean}{std}$

In [8]:
mean=train_data.mean(axis=0)#按照列维度取均值
std=train_data.std(axis=0)

train_data=(train_data-mean)/std
test_data=(test_data-mean)/std

## 数据预处理到此为止，之后我们构造神经网络，从中可以发现：
### 数据量较小，非常容易过拟合，所以我们模型不应该太复杂，用两个隐藏层即可

In [11]:
def build_model():
    model=Sequential()
    model.add(Dense(64,input_shape=(13,),activation='relu'))
    model.add(Dense(64,activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=RMSprop(),loss="mse",metrics=["mae"])
    #因为题目要求的方式是RMSE,mae平方绝对误差
    return model
    

## 众所周知，当数据较小的时候，我们应该采取交叉验证

<img src="imgs/cross_val.png">
<h3>一般而言，我们Fold取10 or 5,当得到的结果的平均水平我们可以接受了，再用所有的数据作为训练集
<h3>构造一个模型，再对test集进行检验

In [12]:
k=4#(emmm,train data we have 404)
num_val_samples=len(train_data)//k #(Python3)
num_epochs=100
all_scores=[]

In [17]:
for i in range(k):
    val_data=train_data[i*num_val_samples:(i+1)*num_val_samples]
    val_targets=train_target[i*num_val_samples:(i+1)*num_val_samples]
    
    partial_train_data=np.concatenate(
                                      [train_data[:i*num_val_samples],
                                      train_data[(i+1)*num_val_samples:]],
                                      axis=0
                                     )
    partial_train_targets=np.concatenate(
                                    [train_target[:i*num_val_samples],
                                      train_target[(i+1)*num_val_samples:]],
                                      axis=0
                                     )
    
    model =build_model()
    model.fit(partial_train_data,partial_train_targets,epochs=num_epochs,batch_size=1,verbose=0)
     
    val_mse,val_mae=model.evaluate(val_data,val_targets,verbose=0)
    
    all_scores.append(val_mae)
    print("第","i+1","Fold的MSE：",val_mse,"MAE:",val_mae)

第 i+1 Fold的MSE： 7.317761421203613 MAE: 1.9767350914454695
第 i+1 Fold的MSE： 23.113035145372447 MAE: 2.858172702317191
第 i+1 Fold的MSE： 14.706525830939265 MAE: 2.6570332003111887
第 i+1 Fold的MSE： 11.49938703763603 MAE: 2.368642826481621


In [18]:
np.mean(all_scores)

2.4651459551388673

In [20]:
model=build_model()
model.fit(train_data,train_target,epochs=80,batch_size=16,verbose=0)

In [21]:
test_mse,test_mae=model.evaluate(test_data,test_target)

102/102 [==============================] - 0s 1ms/step


In [22]:
test_mse

19.131895177504596

In [23]:
np.sqrt(test_mse)

4.37400219221534

#### 比较上面的结果和Kaggle上的排名，训练效果还是可以，能在30%以内，再继续优化的话可能需要在特征工程上做做功课了